# Estimate Roughness from LCDB v5

In [58]:
import rioxarray
import pathlib
import geopandas
import shapely
import geoapis.vector
import dotenv
import os
import numpy
import xarray

In [59]:
lcdb_id = 104400

In [60]:
crs = 2193
base_path = pathlib.Path(r"C:\Local\data\catchments\Waikanae\caches\roughness\land_cover")

In [61]:
x0=1767800; x1=1775500
y0=5469612; y1=5477770
bbox = geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x0,y1],[x1,y1],[x1,y0]])], crs=crs)

### Define Zo Lookup table

In [62]:
# Lookup table values derived from - https://apps.dtic.mil/sti/tr/pdf/ADA274550.pdf
# And also some set from values given from Graeme - will need to replace with something to cite
ZO_LOOKUP = {
    "Transport Infrastructure": 0.00002, # page 30 Blacktop or concrete 
    'Exotic Forest': 0.4, # Page Fairly level wooded country 
    'Built-up Area (settlement)': 0.4,  # Page 30 Village
    'Low Producing Grassland': 0.0075, # page 25 Thick grass, 5 to 6 cm high  
    'River': 0.004, # Value picked by Rose & Alice. To replace from table 
    'Herbaceous Freshwater Vegetation': 0.04, # Value picked by Rose & Alice. To replace from table 
    'Indigenous Forest': 0.4, # Page 26 Fairly level wooded country 
    'Broadleaved Indigenous Hardwoods': 0.4, # Page 26 Fairly level wooded country 
    'Lake or Pond': 0.004,  # Value picked by Rose & Alice. To replace from table 
    'Manuka and/or Kanuka': 0.4, # Page 26 Fairly level wooded country 
    'Gorse and/or Broom': 0.25, # Page 26 Brush, scrub growth, dense 
    'High Producing Exotic Grassland': 0.09, # page 26 Thick grass, 50 cm high 
    'Deciduous Hardwoods': 0.4, # Page 26 Fairly level wooded country 
    'Sand or Gravel': 0.008,  # Value picked by Rose & Alice. To replace from table 
    'Mixed Exotic Shrubland': 0.25, # Page 26 Brush, scrub growth, dense 
    'Surface Mine or Dump': 0.4,  # Value picked by Rose & Alice. To replace from table 
    'Orchard, Vineyard or Other Perennial Crop': 0.31, # Page 28 Citrus orchard 3.2 m 
    'Forest - Harvested': 0.4, # Page 26 Forest clearings, cutover areas 
    'Gravel or Rock': 0.01,  # Value picked by Rose & Alice. To replace from table 
    'Fernland': 0.25, # Page 26 Brush, scrub growth, dense  
    "Matagouri or Grey Scrub": 0.16, # Page 26 Brush, scrub growth, open 
    "Urban Parkland/Open Space": 25, # Page 26 Field, scattered trees, hedges
}

### Load in land cover map

In [80]:
if not (base_path / "lcdb_v5.gpkg").exists():
    dotenv.load_dotenv()
    lris_key = os.environ.get("LRIS_API", None)
    fetcher = geoapis.vector.Lris(key=lris_key,
                                  bounding_polygon=bbox,
                                  verbose=True,
                                 )
    land_cover = fetcher.run(layer=lcdb_id)
    land_cover.to_file(base_path / "lcdb_v5.gpkg")
else:
    land_cover = geopandas.read_file(base_path / "lcdb_v5.gpkg")

### Drop unwanted years

In [81]:
# Clean data
year_to_keep = 2012
years_to_drop = [2001, 1996, 2008, 2018]
columns = ['EditAuthor', 'EditDate']
for year in years_to_drop:
    columns.extend([f"Name_{year}", f"Class_{year}", f"Wetland_{str(year)[-2:]}", f"Onshore_{str(year)[-2:]}"])
land_cover = land_cover.drop(columns=columns)

### Load in DEM

In [82]:
dem = rioxarray.rioxarray.open_rasterio(base_path / ".." / "geofabrics" / "geofabric_4m_with_waterways.nc",
            masked=True,
            parse_coordinates=True,
        ).squeeze("band", drop=True)
if "data_source" in dem.keys():
    dem["data_source"] = dem.data_source.astype(numpy.float32)
if "lidar_source" in dem.keys():
    dem["lidar_source"] = dem.data_source.astype(numpy.float32)
if "z" in dem.keys():
    dem["z"] = dem.z.astype(numpy.float32)

### Map zo

In [83]:
dem["zo"] = xarray.zeros_like(dem.z)
dem.zo.rio.write_crs(crs, inplace=True)
dem.zo.rio.write_nodata(numpy.nan, encoded=True, inplace=True)
for name in land_cover[f"Name_{year_to_keep}"].unique():
    print(f"name {name}, and value {ZO_LOOKUP[name]}")
    dem["zo"] = dem.zo.where(
        dem.zo.rio.clip(land_cover[land_cover[f"Name_{year_to_keep}"]==name].geometry, drop=False).isnull(),
        ZO_LOOKUP[name]
    )
dem["zo"] = dem.zo.where(dem.zo!=0, 0.004)

name Manuka and/or Kanuka, and value 0.4
name Exotic Forest, and value 0.4
name Built-up Area (settlement), and value 0.4
name High Producing Exotic Grassland, and value 0.09
name Low Producing Grassland, and value 0.0075
name River, and value 0.004
name Herbaceous Freshwater Vegetation, and value 0.04
name Indigenous Forest, and value 0.4
name Broadleaved Indigenous Hardwoods, and value 0.4
name Lake or Pond, and value 0.004
name Gorse and/or Broom, and value 0.25
name Urban Parkland/Open Space, and value 25
name Deciduous Hardwoods, and value 0.4
name Sand or Gravel, and value 0.008
name Forest - Harvested, and value 0.4
name Surface Mine or Dump, and value 0.4
name Transport Infrastructure, and value 2e-05
name Mixed Exotic Shrubland, and value 0.25
name Orchard, Vineyard or Other Perennial Crop, and value 0.31
name Gravel or Rock, and value 0.01
name Fernland, and value 0.25
name Matagouri or Grey Scrub, and value 0.16


In [84]:
zo = dem.drop(["data_source", "lidar_source", "z"])
zo.to_netcdf(base_path / f"zo_table_v1_{year_to_keep}.nc")